![uc3m](uc3m.jpg)

# The min cost flow problem

<a href="http://www.est.uc3m.es/nogales" target="_blank">Javier Nogales</a>



## Summary

Model in Pyomo the min cost flow problem in the Google OR-Tools: https://developers.google.com/optimization/flow/mincostflow

![uc3m](mincostflow.png)



    






### Add here the formulation of the problem, including the vector cost and the incidence matrix

You need to solve the problem using the linear-model formulation, i.e. minimize $c'x$ subject to $Ax=b$ and $l\leq x \leq u$ 


## Formulation with Pyomo



### The data



In [1]:
import numpy as np

# The input

start_nodes = [ 0, 0,  1, 1,  1,  2, 2,  3, 4]
end_nodes   = [ 1, 2,  2, 3,  4,  3, 4,  4, 2]
capacities  = [15, 8, 20, 4, 10, 15, 4, 20, 5]
unit_costs  = [ 4, 4,  2, 2,  6,  1, 3,  2, 3]
supplies = [20, 0, 0, -5, -15]

# Incidence matrix (manual form)

#               01 02 12 13 14 23 24  34 42
# A = np.array([[1, 1, 0, 0, 0, 0, 0, 0, 0], # 0
#               [-1,0, 1, 1, 1, 0, 0, 0, 0], # 1
#               [0,-1,-1, 0, 0, 1, 1, 0,-1], # 2
#               [0, 0, 0,-1, 0,-1, 0, 1, 0], # 3
#               [0, 0, 0, 0,-1, 0,-1,-1, 1], # 4
#              ])

# Incidence matrix (automatic form)
A = np.zeros((len(supplies),len(start_nodes)))
def generateA (A,start_nodes,end_nodes):
    count = 0
    for i in zip(start_nodes,end_nodes):
        A[i[0]][count] = 1
        A[i[1]][count] = -1
        count+=1

generateA (A,start_nodes,end_nodes)
c=np.array(unit_costs)
b=np.array(supplies)

### The model

In [2]:
from pyomo.environ import *

# Create an instance of the model
model = ConcreteModel()

# Initialize some ranges for the constraint and Objective definition 
#model.r = RangeSet(1,len(A))
#model.j = RangeSet(1,len(A.T))

r_labels = [r for r in range(len(A))]
j_labels = [j for j in range(len(A.T))]

model.r = Set(initialize = r_labels)
model.j = Set(initialize = j_labels)

#model.r = RangeSet(0,len(A)-1)
#model.j = RangeSet(0,len(A.T)-1)

model.r.pprint()
model.j.pprint()

print(f"{len(model.r)=}")
print(f"{len(model.j)=}")

r : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
j : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    9 : {0, 1, 2, 3, 4, 5, 6, 7, 8}
len(model.r)=5
len(model.j)=9


#### Define the variables

In [3]:
model.flow = Var(model.j, within = NonNegativeReals)
model.flow.pprint()

flow : Size=9, Index=j
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  None :  None : False :  True : NonNegativeReals
      1 :     0 :  None :  None : False :  True : NonNegativeReals
      2 :     0 :  None :  None : False :  True : NonNegativeReals
      3 :     0 :  None :  None : False :  True : NonNegativeReals
      4 :     0 :  None :  None : False :  True : NonNegativeReals
      5 :     0 :  None :  None : False :  True : NonNegativeReals
      6 :     0 :  None :  None : False :  True : NonNegativeReals
      7 :     0 :  None :  None : False :  True : NonNegativeReals
      8 :     0 :  None :  None : False :  True : NonNegativeReals


#### Define the objective function

In [4]:
def objective(model):
    return sum(c[j]*model.flow[j] for j in model.j)

model.objective = Objective(rule=objective,sense=minimize)
model.objective.pprint()

objective : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 4*flow[0] + 4*flow[1] + 2*flow[2] + 2*flow[3] + 6*flow[4] + flow[5] + 3*flow[6] + 2*flow[7] + 3*flow[8]


In [5]:
def flow_balance(model, r, j):
    return sum([A[r][j]*model.flow[j] for j in model.j]) == b[r]

model.flow_balance = Constraint(model.r, model.j, rule=flow_balance)
model.flow_balance.pprint()

flow_balance : Size=45, Index=flow_balance_index, Active=True
    Key    : Lower : Body                                              : Upper : Active
    (0, 0) :  20.0 :                                 flow[0] + flow[1] :  20.0 :   True
    (0, 1) :  20.0 :                                 flow[0] + flow[1] :  20.0 :   True
    (0, 2) :  20.0 :                                 flow[0] + flow[1] :  20.0 :   True
    (0, 3) :  20.0 :                                 flow[0] + flow[1] :  20.0 :   True
    (0, 4) :  20.0 :                                 flow[0] + flow[1] :  20.0 :   True
    (0, 5) :  20.0 :                                 flow[0] + flow[1] :  20.0 :   True
    (0, 6) :  20.0 :                                 flow[0] + flow[1] :  20.0 :   True
    (0, 7) :  20.0 :                                 flow[0] + flow[1] :  20.0 :   True
    (0, 8) :  20.0 :                                 flow[0] + flow[1] :  20.0 :   True
    (1, 0) :   0.0 :           - flow[0] + flow[2] + flow[

In [6]:
def max_capacities(model, constraintList):
    for j, c in enumerate(capacities):
        constraintList.add(expr = model.flow[j] <= c)
    constraintList.pprint()

model.max_capacities = ConstraintList()
max_capacities(model, model.max_capacities)

max_capacities : Size=9, Index=max_capacities_index, Active=True
    Key : Lower : Body    : Upper : Active
      1 :  -Inf : flow[0] :  15.0 :   True
      2 :  -Inf : flow[1] :   8.0 :   True
      3 :  -Inf : flow[2] :  20.0 :   True
      4 :  -Inf : flow[3] :   4.0 :   True
      5 :  -Inf : flow[4] :  10.0 :   True
      6 :  -Inf : flow[5] :  15.0 :   True
      7 :  -Inf : flow[6] :   4.0 :   True
      8 :  -Inf : flow[7] :  20.0 :   True
      9 :  -Inf : flow[8] :   5.0 :   True


In [7]:
# Solution

Solver = SolverFactory('gurobi')
#Solver.options['DualReductions'] = 0
Results = Solver.solve(model)

# Display the results
model.display()


Model unknown

  Variables:
    flow : Size=9, Index=j
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  12.0 :  None : False : False : NonNegativeReals
          1 :     0 :   8.0 :  None : False : False : NonNegativeReals
          2 :     0 :   8.0 :  None : False : False : NonNegativeReals
          3 :     0 :   4.0 :  None : False : False : NonNegativeReals
          4 :     0 :   0.0 :  None : False : False : NonNegativeReals
          5 :     0 :  12.0 :  None : False : False : NonNegativeReals
          6 :     0 :   4.0 :  None : False : False : NonNegativeReals
          7 :     0 :  11.0 :  None : False : False : NonNegativeReals
          8 :     0 :   0.0 :  None : False : False : NonNegativeReals

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 150.0

  Constraints:
    flow_balance : Size=45
        Key    : Lower : Body  : Upper
        (0, 0) :  20.0 :  20.0 :  20.0
 

### Interpretation



The minimum-cost flow problem is one of the most important problems in network optimization and it has many applications and can be thought as being one of building blocks of more complex models.

This problem is particularly interesting as for a balanced network (the sum of supplies is equal to the sum of demand), where all values in the network are integers, is guaranteed to have an integer solution. This is a consequence of the fact that the incidence matrix $A$ of the problem is unimodular, with $det(A) = \pm 1$, then $Ax = b$ with $M$ and $b$ having integer entries means that $x \in \mathbb{Z}$.

In this particular problem supply nodes had $b_r > 0$, intermediate ones had $b_r=0$ and requiring nodes had $b_r < 0$. This is one of the differences between the minimum-cost flow problem and the transportation one: the latter does not have intermediate nodes.

In this problem it is necessary to minimize the total cost of the flow, so we write: